# TGDS Model with Charge as Input Feature

In [ ]:
import context
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import src.data.data_preprocessing as util
import src.models.lstm_model as lstm

tf.compat.v1.set_random_seed(1)

### Set Hyperparameters

In [ ]:
HYPER_PARAMS = {'n_epochs': 15,                   # number of training epochs
                'use_case': 2,                    # use case (1-3)
                'd_sample': 4,                    # subsampling ratio
                'gauss_sigma': 10,                # smoothing factor
                'd_t': 0.25,                      # charge integration factor
                'feature_range_cur_low': -1,      # lower bound of current input feature after scaling
                'feature_range_cur_high': 1,      # upper -||-
                'feature_range_charge_low': -1,   # lower bound of charge input feature after scaling
                'feature_range_charge_high': 1,   # upper -||-
                'feature_range_volt_low': -1,     # lower bound of voltage label after scaling
                'feature_range_volt_high': 1,     # upper -||-
                'boundary_cur_low': -10,          # lower bound of current value range in A
                'boundary_cur_high': 10,          # upper -||-
                'boundary_charge_low': -33.2,     # lower bound of accumulated current value range 
                'boundary_charge_high': 33.2,     # upper -||-
                'boundary_voltage_low': 3.304,    # lower bound of voltage value range in V
                'boundary_voltage_high': 3.982,   # upper -||-
                'n_steps': 100,                   # defines M in the M-to-1 LSTM structure
                'n_features': 2,                  # number of input features
                'n_lstm_units_1': 50,             # number of LSTM units in the first layer
                'alpha_1': 0.1,                   # alpha value for LeakyReLU 
                'n_lstm_units_2': 20,             
                'alpha_2': 0.1,                   
                'n_dense_units': 10,              # number of dense units in the output layer
                'activation_output_layer': 'tanh',# output activation function
                'dropout': 0.2,                   # dropout factor
                'learning_rate': 0.001,           # learning rate
                'optimizer': 'Adam',              # optimizer
                'metric': 'mae',                  # performance metric during training 
                'loss_funcs': ['mse'],            # loss functions
                'lambda_mse': 1,                  # loss function weighting factor
                'lambda_apx': 0.1,
                'lambda_mon': 0.1,
                }

### Prepare Training/Validation/Test Data
Training data can be modified to account for specific use cases. The prepare_data function is fed with the required profiles and will preprocess the sequence. Three use cases are predefined:
###### 1. Use Case: Train and Test data are built from equal profiles
 The purpose is to find out if the network is able to accurately reproduce the seen profiles
###### 2. Use Case: Train and Test data are of equal value range but different profiles
The purpose is to find out if the network can abstract onto unseen profiles with similar characteristics
###### 3. Use Case: Train and Test data are of different distributions
The purpose is to find out if the network can abstract onto new value ranges and unseen profiles

In [ ]:
train_profiles_usecase_1 = ['Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A', 'Profile 10A']
train_profiles_usecase_2 = ['Profile 10A', 'Profile 10A Run 040618', 'Profile 10A Run 070618', 'Profile 10A Run 080618', 'Profile 10A Run 040618', 'Profile 10A Run 070618_3', 'Profile 10A Run 070618_2', 'Profile 10A Run 070618']
train_profiles_usecase_3 = ['Profile 10A']

val_profiles_usecase_1 = ['Profile 10A']
val_profiles_usecase_2 = ['Profile 10A']
val_profiles_usecase_3 = ['Profile 10A']

test_profiles_usecase_1 = ['Profile 10A']
test_profiles_usecase_2 = ['Profile 10A 3x']
test_profiles_usecase_3 = ['Profile -10A']

if HYPER_PARAMS['use_case'] == 1:
    X_train, y_train, scalers_train = util.prepare_current_charge_input(HYPER_PARAMS, train_profiles_usecase_1, 0, 4)
    X_validation, y_validation, _ = util.prepare_current_charge_input(HYPER_PARAMS, val_profiles_usecase_1, 0, 4)
    X_test, y_test, _ = util.prepare_current_charge_input(HYPER_PARAMS, test_profiles_usecase_1, 0, 4)
elif HYPER_PARAMS['use_case'] == 2:
    X_train, y_train, scalers_train = util.prepare_current_charge_input(HYPER_PARAMS, train_profiles_usecase_2, 0, 4)
    X_validation, y_validation, _ = util.prepare_current_charge_input(HYPER_PARAMS, val_profiles_usecase_2, 0, 4)
    X_test, y_test, _ = util.prepare_current_charge_input(HYPER_PARAMS, test_profiles_usecase_2, 0, 4)
elif HYPER_PARAMS['use_case'] == 3:
    X_train, y_train, scalers_train = util.prepare_current_charge_input(HYPER_PARAMS, train_profiles_usecase_3, 0, 4)
    X_validation, y_validation, _ = util.prepare_current_charge_input(HYPER_PARAMS, val_profiles_usecase_3, 0, 4)
    X_test, y_test, _ = util.prepare_current_charge_input(HYPER_PARAMS, test_profiles_usecase_3, 0, 4)

# plt.plot(y_train)

### Prepare Data for Use Cases

In [ ]:
test_profiles_usecase_1 = ['Profile 10A']
test_profiles_usecase_2 = ['Profile 10A 3x']
test_profiles_usecase_3 = ['Profile -10A']

X_test_1, y_test_1, _ = util.prepare_current_charge_input(HYPER_PARAMS, test_profiles_usecase_1, 0, 4)
X_test_2, y_test_2, _ = util.prepare_current_charge_input(HYPER_PARAMS, test_profiles_usecase_2, 0, 4)
X_test_3, y_test_3, _ = util.prepare_current_charge_input(HYPER_PARAMS, test_profiles_usecase_3, 0, 4)

### Initialize and Train Model

In [ ]:
lstm = lstm.Model()
lstm.initialize(HYPER_PARAMS)

In [ ]:
_, fig = lstm.train(X_train, y_train, scalers_train)

# save model and hyperparameters
MODEL_ID = str(np.random.randint(10000))

lstm.model.save('../../../models/TGDS/' + str(MODEL_ID))
np.save('../../../models/TGDS/' + str(MODEL_ID) + '/hyperparameters', HYPER_PARAMS)
fig.savefig('../../../reports/figures/theory_guided_charge-' + str(MODEL_ID) + '-learning_curve.png')

### Test Model

In [ ]:
yhat_train_unscaled, _, _, _, _, fig = lstm.test(X_train, y_train, X_validation, y_validation, X_test, y_test, scalers_train)

# save plots and predicted sequences
np.save('../../../models/TGDS/' + str(MODEL_ID) + '/predictions', yhat_train_unscaled)
fig.savefig('../../../reports/figures/theory_guided_charge-' + str(MODEL_ID) + '-validation&test_profiles.png')

In [ ]:
lstm.test_usecases(X_train, y_train, X_test_1, y_test_1, X_test_2, y_test_2, X_test_3, y_test_3, scalers_train)